In [1]:
from gym_model import Gym

In [2]:
gym = Gym(4)
for _ in range(5):
    gym.step()
    print("-"*10)

Agent 3 @ (0, 0) (SEARCHING)
Agent 1 @ (0, 0) (SEARCHING)
Agent 2 @ (0, 0) (SEARCHING)
Agent 0 @ (0, 0) (SEARCHING)
----------
Agent 0 @ (0, 1) (SEARCHING)
Agent 2 @ (0, 1) (SEARCHING)
Agent 3 @ (0, 1) (SEARCHING)
Agent 1 @ (0, 1) (SEARCHING)
----------
Agent 2 @ (0, 2) (SEARCHING)
Agent 0 @ (0, 2) (SEARCHING)
Agent 3 @ (0, 2) (SEARCHING)
Agent 1 @ (0, 0) (SEARCHING)
----------
Agent 3 @ (0, 1) (SEARCHING)
Agent 1 @ (0, 1) (SEARCHING)
Agent 0 @ (0, 3) (SEARCHING)
Agent 2 @ (0, 1) (SEARCHING)
----------
Agent 2 @ (0, 0) (SEARCHING)
Agent 0 @ (0, 2) (SEARCHING)
Agent 1 @ (0, 0) (SEARCHING)
Agent 3 @ (0, 0) (SEARCHING)
----------


In [1]:
from mesa.experimental import JupyterViz
from gym_model import Gym

model_params = {
    "num_trainees": {
        "type": "SliderInt",
        "value": 2,
        "label": "Number of agents:",
        "min": 1,
        "max": 50,
        "step": 1,
    },
}


from gym_agent import GymRat

page = JupyterViz(
    Gym,
    model_params,
    measures=[],
    name="Test gym",
    agent_portrayal=lambda agent: agent.portrayal,
    space_drawer="default",
    # space_drawer=GymDrawer,
)

page

Cannot show ipywidgets in text

Agent 1 @ (0, 0) (SEARCHING)
Agent 0 @ (0, 0) (SEARCHING)
Agent 0 @ (0, 1) (SEARCHING)
Agent 1 @ (0, 1) (SEARCHING)
Agent 1 @ (0, 2) (SEARCHING)
Agent 0 @ (0, 0) (SEARCHING)
Agent 0 @ (0, 1) (SEARCHING)
Agent 1 @ (0, 1) (SEARCHING)
Agent 0 @ (0, 0) (SEARCHING)
Agent 1 @ (0, 2) (SEARCHING)
Agent 0 @ (0, 1) (SEARCHING)
Agent 1 @ (0, 1) (SEARCHING)
Agent 0 @ (0, 0) (SEARCHING)
Agent 1 @ (0, 2) (SEARCHING)
Agent 1 @ (0, 1) (SEARCHING)
Agent 0 @ (0, 1) (SEARCHING)
Agent 1 @ (0, 2) (SEARCHING)
Agent 0 @ (0, 2) (SEARCHING)
Agent 0 @ (0, 3) (SEARCHING)
Agent 1 @ (0, 1) (SEARCHING)
Agent 0 @ (0, 2) (SEARCHING)
Agent 1 @ (0, 0) (SEARCHING)
Agent 0 @ (0, 3) (SEARCHING)
Agent 1 @ (0, 1) (SEARCHING)
Agent 0 @ (0, 2) (SEARCHING)
Agent 1 @ (0, 2) (SEARCHING)
Agent 0 @ (0, 3) (SEARCHING)
Agent 1 @ (0, 1) (SEARCHING)
Agent 0 @ (0, 2) (SEARCHING)
Agent 1 @ (0, 0) (SEARCHING)
Agent 0 @ (0, 1) (SEARCHING)
Agent 1 @ (0, 1) (SEARCHING)
Agent 0 @ (0, 2) (SEARCHING)
Agent 1 @ (0, 0) (SEARCHING)
Agent 1 @ (0, 

In [8]:
import gym_agent

print(list(gym_agent.State))

[<State.SEARCHING: 1>, <State.WORKING_OUT: 2>, <State.RESTING: 3>]


In [ ]:
import solara
from matplotlib.figure import Figure
import numpy as np

@solara.component
def GymDrawer(model: Gym, agent_portrayal):
    print("GymDrawer called")
    space_fig = Figure()
    space_ax = space_fig.subplots()
    agents = model.agent_layer
    _draw_grid(agents, space_ax, agent_portrayal)

    equipment_x = []; equipment_y = []
    for (row, col), val in np.ndenumerate(model.equipment_layer):
        if val is not None:
            equipment_x.append(row)
            equipment_y.append(col)
    space_ax.scatter(equipment_x, equipment_y, color="black", s=50, marker="2")

    # space_ax.set_axis_off()
    space_ax.set_xlim(0, 10)
    space_ax.set_ylim(0, 10)
    solara.FigureMatplotlib(space_fig, format="png", dependencies=None)


def _draw_grid(space, space_ax, agent_portrayal):
    def portray(g):
        x = []
        y = []
        s = []  # size
        c = []  # color
        for i in range(g.width):
            for j in range(g.height):
                content = g._grid[i][j]
                if not content:
                    continue
                if not hasattr(content, "__iter__"):
                    # Is a single grid
                    content = [content]
                for agent in content:
                    data = agent_portrayal(agent)
                    x.append(i)
                    y.append(j)
                    if "size" in data:
                        s.append(data["size"])
                    if "color" in data:
                        c.append(data["color"])
        out = {"x": x, "y": y}
        if len(s) > 0:
            out["s"] = s
        if len(c) > 0:
            out["c"] = c
        print("portray:", out)
        return out

    space_ax.scatter(**portray(space))

